# Direct Radio Poll Test

In [ ]:
%autosave 0

In [ ]:
from __future__ import print_function
from builtins import *                  # python3 types
from time import sleep
import struct as pystruct
from binascii import hexlify
import sys
import os.path
import datetime

In [ ]:
sys.path.insert(0,"../tagnet")
from tagnet.tagmessages import TagMessage, TagPoll, TagGet, TagPut, TagDelete, TagHead
from tagnet.tagnames import TagName
from tagnet.tagtlv import TagTlv, TagTlvList, tlv_types

In [ ]:
sys.path.insert(0,"../tagfuse")
sys.path.insert(0,"../si446x/")
from tagfuse.radioutils import payload2values, radio_receive_msg, radio_send_msg
from tagfuse.radioutils import radio_start, radio_config, radio_show_config, radio_poll
from tagfuse.radioutils import radio_get_rtctime, radio_set_rtctime

In [ ]:
from tagfuse.tagfuseargs import process_cmd_args, get_cmd_args
import tagfuse.tagfuseargs
tagfuse.tagfuseargs.global_args
tagfuse.tagfuseargs.global_args=None
#sys.argv = ['tagfuse.py', '~/tags', '--disable_sparse']
sys.argv = ['tagfuse.py', '/home/pi/tag_store']
process_cmd_args()
get_cmd_args()
from tagfuse.TagFuseTree import TagFuseTagTree

In [ ]:
sys.path

In [ ]:
from tagfuse.tagfuseargs import __version__
'tagfuse', __version__

In [ ]:
print('Test Start Time: {}'.format(datetime.datetime.now()))
#print('Si446x Radio Device Driver Version: {}'.format(si446x_device_version()))

##  Start up Radio

In [ ]:
#radio = radio_start()
from si446x import Si446xRadio
radio=Si446xRadio(0)
if (radio == None):
    raise RuntimeError('radio_start: could not instantiate radio')
radio.unshutdown()
radio.write_config()

In [ ]:
from si446x import get_config_wds, get_name_wds
get_name_wds()

In [ ]:
radio.write_config()

In [ ]:
radio.config_frr()

In [ ]:
radio_show_config(radio.dump_radio())

### Set Defaults

In [ ]:
# default paramters
MAX_WAIT            = 1
MAX_RECV            = 2
MAX_PAYLOAD         = 254
MAX_RETRIES         = 3
RADIO_POWER         = 20
SHORT_DELAY         = 1

### Poll for Tags

In [ ]:
STOP


In [ ]:
payload, rssi, sstatus, rstatus = radio_poll(radio)
payload, rssi

In [ ]:
d = {}
d[hexlify(payload[0][0])] = TagFuseTagTree(radio)

In [ ]:
d

In [ ]:
b = payload[0]
len(b), b

In [ ]:
for x in range(len(b)):
    print(b[x])

In [ ]:
errors = 0
for i in range(200000):
    payload, rssi, sstatus, rstatus = radio_poll(radio)
    if not payload:
        errors += 1
    #print(payload, rssi)
    if (i % 10) == 0:
        print(i, errors, rssi, payload)
print('errors', errors)

In [ ]:
import platform
platform.node()

In [ ]:
TagTlv(tlv_types)

In [ ]:
rssi, payload

In [ ]:
'send', sstatus, 'receive', rstatus

In [ ]:
radio.trace.display(radio.trace.filter(count=-10))

## Tag UTC Time

In [ ]:
STOP

In [ ]:
params = radio_get_rtctime(radio)
params[0]

In [ ]:
ut = datetime.datetime.utcnow()
radio_set_rtctime(radio, ut)
ut

## Listen for packets

In [ ]:
for i in range(200000):
    rsp_msg, rssi, status = radio_receive_msg(radio, MAX_RECV, 3000)
    if rsp_msg:
        print(datetime.datetime.now(), rssi, len(rsp_msg), hexlify(rsp_msg))
        #rsp_obj = TagMessage(rsp_msg)
        #        print(rsp_obj.header.options.param.error_code)
        #        print(rsp_obj.payload)
        #if rsp_obj.payload:
        #    print(rsp_obj.header, rsp_obj.name, rsp_obj.payload)

## Get Chip Status

In [ ]:
STOP

In [ ]:
print(radio.get_chip_status())

## Interactive Group Properties

In [ ]:
STOP

In [ ]:
interact(si446x_device_group_fetch_and_decode, group=radio_config_group_ids.encoding)

## Interactive  Command Status Responses

In [ ]:
interact(si446x_device_command_fetch_and_decode, cmd=radio_status_cmd_ids.encoding)

In [ ]:
from datetime import datetime
datetime.now()